@Farhan: This is a copy of the file from P23-18 Transitiemanager we discussed in 19/08/2024. You can used snippets from it to create a workbook for the creation of a zoning layer for Hasselt (or convert this one into the workbook for Hasselt).

# Extract buildings from OSM

Install required libraries using the Conda conda-forge channel
````bash
conda install -c conda-forge jupyter
conda install -c conda-forge requests
````

Download relevant files to `./data`

In [ ]:
import os
import requests

os.makedirs("./data", exist_ok=True)  # create data folder if not exist


# Download file
def dlfile(furl, fname):
    if not os.path.isfile(fname):
        print(f"File {fname} does not exist.")
        print(f"Downloading from {furl}")

        chunk_size = 4096
        with requests.get(furl, stream=True) as r:
            with open(fname, "wb") as f:
                for chunk in r.iter_content(chunk_size):
                    if chunk:
                        f.write(chunk)
    else:
        print(f"File {fname} is available.")


print("Acquiring required data")

# Statistische sectoren
SSUrl = "https://statbel.fgov.be/sites/default/files/files/opendata/Statistische%20sectoren/sh_statbel_statistical_sectors_3812_20230101.shp.zip"
SSFile = "./data/sh_statbel_statistical_sectors_3812_20230101.shp.zip"
dlfile(SSUrl, SSFile)

# Bevolking per statistische sector
BVSSUrl = "https://statbel.fgov.be/sites/default/files/files/opendata/bevolking/sectoren/OPEN%20DATA_SECTOREN_2019.xlsx"
BVSSFile = "./data/OPEN_DATA_SECTOREN_2019.xlsx"
dlfile(BVSSUrl, BVSSFile)

# OSM data voor België
OSMUrl = "https://download.geofabrik.de/europe/belgium-latest.osm.pbf"
OSMFile = "./data/belgium.osm.pbf"
dlfile(OSMUrl, OSMFile)

## (Optioneel/test) Cut de oorspronkelijke pbf bestanden in kleinere
Gebruik Osmium.

````bash
pip install osmium
````

Cut obv boudning box.
Bounding box tool: https://boundingbox.klokantech.com/ (selecteer CSV).

- AVBL (Antwerpen, Vlaams-Brabant, Limburg): 3.85,50.7121,6.1183,51.5039
- Hasselt: 5.265463,50.894442,5.452711,50.956291

````bash
# AVBL
osmium extract --bbox 3.85,50.7121,6.1183,51.5039 ./data/belgium.osm.pbf -o ./data/AVBL.osm.pbf
# Hasselt
osmium extract --bbox 5.265463,50.894442,5.452711,50.956291 ./data/belgium.osm.pbf -o ./data/Hasselt.osm.pbf
````


In [ ]:
# Gebruik voor tests het beperkte bestand
!osmium extract --overwrite --bbox 5.265463,50.894442,5.452711,50.956291 ./data/belgium.osm.pbf -o ./data/Hasselt.osm.pbf
OSMFile = "./data/Hasselt.osm.pbf" 
OSMUrl = ""

!osmium extract --overwrite --bbox 3.85,50.7121,6.1183,51.5039 ./data/belgium.osm.pbf -o ./data/AVBL.osm.pbf
OSMFile = "./data/AVBL.osm.pbf" 
OSMUrl = ""


# Create shp files with populatie

Use official data from the statistical center and start from the SS to create files for DLG and GEM level.

In [ ]:
import geopandas as gpd

# gpd.options.io_engine = "pyogrio"
# import fiona

# https://geopandas.org/en/v0.14.0/docs/user_guide/io.html
#     where="CNIS_PROVI='70000'",
#     where="(CNIS_PROVI IN ('70000', '20001', '10000')) OR ( CNIS_PROVI IS NULL)",
gdf_SS = gpd.read_file(
    f"zip://{SSFile}!sh_statbel_statistical_sectors_3812_20230101.shp",
    where="(CNIS_PROVI IN ('70000', '20001', '10000')) OR ( CNIS_PROVI IS NULL)",
    columns=[
        "CS01012023",
        "CNIS_PROVI",
        "T_SEC_NL",
        "CNIS_REGIO",
        "T_ARRD_NL",
        "T_MUN_NL",
        "CNIS5_2023",
        "C_NIS6",
        "T_NIS6_NL",
    ],
)
# Pyogrio: https://pyogrio.readthedocs.io/en/latest/introduction.html#read-a-subset-of-columns
# gdf = gdf[["CS01012023",
#        "CNIS_PROVI",
#        "T_SEC_NL",
#        "CNIS_REGIO",
#        "T_ARRD_NL",
#        "T_MUN_NL", "geometry"
#    ]]
print(gdf_SS.columns)
gdf_SS.head()

Load excel file with population per SS in Pandas dataframe

````bash
conda install -c conda-forge openpyxl
````

In [ ]:
import pandas as pd

# bvss = pd.read_excel(BVSSFile)
bvss = pd.read_excel(BVSSFile, usecols=["POPULATION", "CD_SECTOR"])
bvss.head()

Join bvss dataframe with population with geodataframe with SS data to add population data per SS to the SS shapefile.

In [ ]:
gdf_SS = gdf_SS.merge(bvss, left_on="CS01012023", right_on="CD_SECTOR")
gdf_SS.head()
# drop unneeded foreign key column before export
gdf_SS = gdf_SS.drop(["CD_SECTOR"], axis="columns")
gdf_SS.to_file("./data/SS_pop.shp")

Create shapefiles for DLG and GEM by aggregation of SS.

In [ ]:
# Aggregate SS to DLG
gdf_DLG = gdf_SS.dissolve(
    by="C_NIS6",  # Do not aggregate on T_NIS6_NL as it cannot be an index: duplicate names!
    aggfunc={  # Only listed columns are exported/aggregated
        "CNIS_PROVI": "first",
        "CNIS_REGIO": "first",
        "T_ARRD_NL": "first",
        "T_MUN_NL": "first",
        "T_NIS6_NL": "first",
        "CNIS5_2023": "first",
        "POPULATION": "sum",
    },
).reset_index()  # reset newly created C_NIS6 index to a regular column
gdf_DLG.to_file("./data/DLG_pop.shp")

# Aggregate DLG to GEM
gdf_GEM = gdf_DLG.dissolve(
    by="CNIS5_2023",  # Do not aggregate on T_MUN_NL as it is a label and not necessarily unique
    aggfunc={  # Only listed columns are exported/aggregated
        "CNIS_PROVI": "first",
        "CNIS_REGIO": "first",
        "T_ARRD_NL": "first",
        "POPULATION": "sum",
        "T_MUN_NL": "first",
    },
).reset_index()  # reset newly created CNIS5_2023 index to a regular column
gdf_GEM.to_file("./data/GEM_pop.shp")

## SS per DLG to obtain a x_pct % of the population

Use pivot_table on gdf with the SS and the population. We count how many SS, ordered from large to low population are needed to cover x_pct % of the population in that DLG.

This value is used later as the minimal amount of clusters to create for that DLG.

https://www.codium.ai/blog/pandas-pivot-tables-a-comprehensive-guide-for-data-science/

````bash
mamba install -c conda-forge geopy
````

In [ ]:
# Extrema:
# x_pct = 0   -> No min required population represented in selected SS => 1 cluster for DLG
# x_pct = 100 -> Full population needs to be represented in selected SS => # clusters = # SS in DLG
#                Note: # clusters can be < # SS in DLG if there are SS with POPULATION = 0
x_pct = 66

import numpy as np
from shapely.ops import unary_union, nearest_points

from geopy.geocoders import Photon
import pyproj
from shapely.geometry import Point
from shapely.ops import transform

wgs84 = pyproj.CRS("EPSG:4326")

photon_url = "imobwww.uhasselt.be/photon"
geolocator = Photon(user_agent="Python", domain=photon_url)
location = geolocator.geocode("Dusart Hasselt")
print(location.address)
print((location.latitude, location.longitude))
project = pyproj.Transformer.from_crs(wgs84, gdf_SS.crs, always_xy=True).transform
dusart = Point(location.longitude, location.latitude)
print(dusart)
dusart = transform(project, Point(location.longitude, location.latitude))
print(dusart)

location2 = geolocator.geocode("begijnenstraat diest")
print(location2.address)
print((location2.latitude, location2.longitude))
# project = pyproj.Transformer.from_crs(wgs84, gdf_SS.crs, always_xy=True).transform
diepenbeek = Point(location2.longitude, location2.latitude)
print(diepenbeek)
diepenbeek = transform(project, Point(location2.longitude, location2.latitude))
print(diepenbeek)

print(diepenbeek.distance(dusart))


# Get the number of the largest SS required to obtain x_pct of the POPULATION
# -np.sort(-x) sorts the series descending
# np.cumsum(-np.sort(-x)) generates total numbers of POPULATION by incrementally adding largest SS
# np.sum(x) = total inhabitants in DLG
# x_pct*np.sum(x)/100 : number of inhabitants in DLG that consitutes x_pct of total in DLG
# np.cumsum(-np.sort(-x))-x_pct*np.sum(x)/100 : becomes 0 for the n largest SS that cover x_pct of pop in DLG
# np.abs(...) becomes 0 for the element in the series for which the cumsum becomes x_pct of pop in DLG
# np.argmin( ...) computes the 0-based index of the value closest by 0
# ... + 1 : convert 0-based index to number of SS
def xpct(x):
    return np.argmin(np.abs(np.cumsum(-np.sort(-x)) - x_pct * np.sum(x) / 100)) + 1


def mindist(x):
    return dusart.distance(unary_union(x)) / 1000


# def mindist2(x): return dusart.distance(unary_union(x))
# def mindist(x): return min(dusart.distance(x))

# first test: compute the mean population ...
DLG_pivot_gdf = gdf_SS.pivot_table(
    index="C_NIS6",
    values=["POPULATION", "geometry", "T_NIS6_NL"],
    aggfunc={
        "POPULATION": ["count", xpct],
        "geometry": [mindist],
        "T_NIS6_NL": "first",
    },
)
DLG_pivot_gdf = DLG_pivot_gdf.rename(
    columns={"xpct": f"n_pct_{x_pct}", "first": "T_NIS6_NL"}
)
# Flatten multi level column index
DLG_pivot_gdf.columns = [a[-1] for a in DLG_pivot_gdf.columns.to_flat_index()]
# Reset Index to get the Name column back
DLG_pivot_gdf = DLG_pivot_gdf.reset_index()
print(DLG_pivot_gdf)
print(
    DLG_pivot_gdf.loc[
        DLG_pivot_gdf["T_NIS6_NL"].isin(["DIEPENBEEK", "HASSELT", "DIEST"])
    ]
)
print(gdf_SS.crs)

# Plot histogram with the distances
ax = DLG_pivot_gdf.plot.hist(column="mindist")
ax.set_title("Distance from Dustart to DLGs")
ax.set_xlabel("Distance (Kilometer)")

## Extract building data from pbf file

Maak een polygoon voor Limburg om mee te filteren: LimburgGrenzen.
Lees de gebouwen in.
Centroids

In [ ]:
## First try: Pyrosm. Does not scale well (memory consumption)
# from pyrosm import OSM, get_data
#
# osm = OSM(OSMFile)
# buildings = osm.get_buildings()
# buildings.plot()

Use Osmium to process OSM pbf file

````bash
conda install -c conda-forge osmium-tool
# Besides osmium-tool, osmium itself needs to be installed as well
pip install osmium
````
https://max-coding.medium.com/extracting-open-street-map-osm-street-data-from-data-files-using-pyosmium-afca6eaa5d00

https://github.com/osmcode/pyosmium/blob/master/examples/amenity_list.py

https://stackoverflow.com/questions/10715965/create-a-pandas-dataframe-by-appending-one-row-at-a-time


In [ ]:
# Sample code counting the nodes in the OSMFile pbf file

import osmium


class CounterHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.num_nodes = 0

    def node(self, n):
        self.num_nodes += 1


h = CounterHandler()
h.apply_file(OSMFile)
print("Number of nodes: %d" % h.num_nodes)

In [ ]:
# Extract buildings from OSMFile pbf file

import osmium
import shapely.wkt as wktlib
import geopandas

# A global factory that creates WKB from a osmium geometry
wktfab = osmium.geom.WKTFactory()

# create transformer object for Shapely object crs transformation from
# .. -> ... needed for the area computation
# see docs: https://shapely.readthedocs.io/en/latest/manual.html#shapely.ops.transform
import pyproj
from shapely.ops import transform

ss_crs = gdf_SS.crs  # use the orthonormal projection system from the SS layer
project = pyproj.Transformer.from_crs(
    pyproj.CRS("EPSG:4326"), ss_crs, always_xy=True
).transform


class CounterHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.num_nodes = 0
        self.building_values = []
        self.amenity_values = []
        self.buildings = []
        self.building_area = []

    def area(self, n):
        if "building" in n.tags:
            try:
                wkt = wktfab.create_multipolygon(n)
                self.num_nodes += 1
                area = wktlib.loads(wkt)
                # print(n.tags["building"])
                self.buildings.append(area.centroid)
                self.building_values.append(n.tags.get("building"))
                #                self.building_values.update([k for k, v in n.tags])
                #                if "amenity" in n.tags:
                self.amenity_values.append(n.tags.get("amenity"))
                self.building_area.append(
                    transform(project, area).area
                )  # transform before computing area (inefficient!)
                if self.num_nodes % 250000 == 0:  # report one in 250000 buildings
                    print(f"{area.centroid} : {n.tags}")
                    print(f"Building: {n.tags.get('building')}")
                    print(f"Amenity: {n.tags.get('amenity')}")
            except:
                print("Trouble with polygons ...")


h = CounterHandler()
h.apply_file(OSMFile, locations=True)
print("Number of nodes: %d" % len(h.buildings))
print(f"Buildings: {h.building_values}")
print(f"Amenities: {h.amenity_values}")

## Write some statistics on the imported data

https://www.tutorialspoint.com/how-to-count-occurrences-of-specific-value-in-pandas-column 

This does not work for Python series directly, rather, we first convert to a Pandas Series object and use the values_counts() method.

Note: Alternative is to call the values_counts method later on the columns of the blding_centroids GeoDataFrame.

In [ ]:
import pandas as pd
import openpyxl

building_value_counts = pd.Series(h.building_values).value_counts()
amenity_value_counts = pd.Series(h.amenity_values).value_counts()

print(f"Buildings values: {building_value_counts}")
print(f"Amenities values: {amenity_value_counts}")

# write results to excel file
with pd.ExcelWriter("./data/tag_stats.xlsx") as writer:
    building_value_counts.to_excel(writer, "building")
    amenity_value_counts.to_excel(writer, "amenity")

## Write shapefile with centroids to file

In [ ]:
import geopandas as gpd


print(h.building_values)

# bldng_centroids = gpd.GeoDataFrame( geometry=h.buildings, crs="EPSG:4326")
# bldng_centroids = gpd.GeoDataFrame({'building': h.building_values, 'amenity': h.amenity_values, 'geometry': gpd.GeoSeries(h.buildings)}, crs="EPSG:4326")
bldng_centroids = gpd.GeoDataFrame(
    {
        "building": h.building_values,
        "amenity": h.amenity_values,
        "area": h.building_area,
        "geometry": h.buildings,
    },
    geometry="geometry",
    crs="EPSG:4326",
)

bldng_centroids = bldng_centroids.to_crs(gdf_SS.crs)  # reproject to SS shapefile crs

bldng_centroids.to_file("./data/bldng_centroids.shp")

# Count the OSM points in each of the statistical sectors
Based on the following example: https://medium.com/@nygeog/data-science-methods-focus-geoprocessing-with-geopandas-using-spatial-joins-counting-points-e42d1b36d758

Note: this merely counts the number of points. If we can have points belonging to different categories (e.g. for different amenities), we can use the Pandas pivot table approach as in: https://gis.stackexchange.com/questions/306674/geopandas-spatial-join-and-count


In [ ]:
# Join dataframes
joined_gdf = gdf_SS.sjoin(
    bldng_centroids,
    how="inner",
    predicate="intersects",
)
# Groepeer op SS en tel
ss_count_bldngs = joined_gdf.groupby(
    ["CS01012023"],
    as_index=False,
)[
    "building"
].count()  # this counts the non Null values, so choose column that has no Null
ss_count_bldngs = ss_count_bldngs.rename(columns={"building": "bldng_cnt"})
# ss_count_bldngs.columns = ['building', 'bldng_cnt']  # column rename alternative
# print(ss_count_bldngs.head())

# Merge dataframe with counts per SS back into the dataframe with the SS
# print(gdf.head())
gdf_cnts = gdf_SS.merge(
    ss_count_bldngs,
    on="CS01012023",
    how="left",
)
# print(gdf_cnts.head())

# Cleanup
# Note: all SS outside the SS got NaN since no matching value was found in the count df
# Also, SS without buildings will be missing; set NaN -> 0
gdf_cnts["bldng_cnt"].fillna(
    0,
    inplace=True,
)
# NaN has no integer representation, so the bldng_cnt column is float
# As they are counts, convert to int
gdf_cnts = gdf_cnts.astype({"bldng_cnt": "int"})

# print(gdf_cnts.head())
gdf_cnts.to_file("./data/SS_pop_bldng_cnts.shp")

Make a plot using Matplotlib

In [ ]:
###import matplotlib.pyplot as plt
###%matplotlib inline
###
###gdf_cnts.plot(
###    column='bldng_cnt',
###    figsize=(8, 5),
###    cmap='magma',
###    legend=True,
###)
###plt.title('Number of OSM buildings per Statistical Sector');

In [ ]:
### import folium
###
### # See: https://vverde.github.io/blob/interactivechoropleth.html
###
### #map = folium.Map(location=[50.93, 5.36], tiles="OpenStreetMap", zoom_start=13)
### #map
###
### # Compute center of data
### gdf_cnts = gdf_cnts.to_crs('EPSG:4326')
### x_map=gdf_cnts.centroid.x.mean()
### y_map=gdf_cnts.centroid.y.mean()
### print(x_map,y_map)
###
### mymap = folium.Map(location=[y_map, x_map], zoom_start=11, tiles=None)
### folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)
### #myscale = (gdf_cnts['bldng_cnt'].quantile((0,0.25,0.5,0.75,1))).tolist()
### #print(myscale)
### folium.Choropleth(
###     geo_data=gdf_cnts,
###     name='Choropleth',
###     data=gdf_cnts,
###     columns=['CS01012023','bldng_cnt'],
###     key_on="feature.properties.CS01012023",
###     fill_color='YlGn',
###     scheme='Quantiles',
### #    threshold_scale=myscale,
###     fill_opacity=0.5,
###     line_opacity=0.2,
###     legend_name='Number of buildings in Statistical Sector',
###     smooth_factor=0
### ).add_to(mymap)
### mymap
###

In [ ]:
### # alternative choropleth map
### # See: https://python-visualization.github.io/folium/latest/advanced_guide/colormaps.html for color options
### import branca.colormap as cm
### colormap = cm.linear.RdPu_04.to_step(data=gdf_cnts['bldng_cnt'], n=5, method='linear')
### colormap
###
### # Compute center of data
### gdf_cnts = gdf_cnts.to_crs('EPSG:4326')
### x_map=gdf_cnts.centroid.x.mean()
### y_map=gdf_cnts.centroid.y.mean()
### print(x_map,y_map)
###
### mymap = folium.Map(location=[y_map, x_map], zoom_start=11, tiles=None)
### folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)
###
### colormap.caption = "Number of buildings in Statistical Sector"
### style_function = lambda x: {"weight":0.5,
###                             'color':'black',
###                             'fillColor':colormap(x['properties']['bldng_cnt']),
###                             'fillOpacity':0.75}
### highlight_function = lambda x: {'fillColor': '#000000',
###                                 'color':'#000000',
###                                 'fillOpacity': 0.50,
###                                 'weight': 0.1}
### NIL=folium.features.GeoJson(
###         gdf_cnts,
###         style_function=style_function,
###         control=False,
###         highlight_function=highlight_function,
###         tooltip=folium.features.GeoJsonTooltip(fields=['CS01012023','bldng_cnt','POPULATION'],
###             aliases=['SSID','# buildings','Population'],
###             style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
###             sticky=True
###         )
###     )
### colormap.add_to(mymap)
### mymap.add_child(NIL)
### mymap

# Clustering van de gebouwen in groepen

https://samdotson1992.github.io/SuperGIS/blog/k-means-clustering/
https://darribas.org/gds15/content/labs/lab_08.html

https://gis.stackexchange.com/questions/309087/how-to-create-a-geodataframe-points-grid-from-a-numpy-mgrid

https://stackoverflow.com/questions/50971914/what-is-the-most-efficient-way-to-convert-numpy-arrays-to-shapely-points

Selecteer eerst enkel gebouwen in Halen (T_NIS6_NL="HALEN")


In [ ]:
# Selecteer Halen als gebied om gebouwen mee te selecteren
# gdf_halen = gdf_SS[gdf_SS.T_MUN_NL == "Hasselt"].dissolve(by='T_MUN_NL')
# Fix issues met Memory leak op Windows
# https://stackoverflow.com/questions/69596239/how-to-avoid-memory-leak-when-dealing-with-kmeans-for-example-in-this-code-i-am

# Use slugify to remove invalid characters from autogenerated paths
# mamba install -c conda-forge python-slugify
from slugify import slugify
import os

os.environ["OMP_NUM_THREADS"] = "1"

from colorama import Fore
from colorama import Style
from shapely.geometry import Point

import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import math

# Store the centroids for the cluster, including their cluster ID (cluster)
# Note that clusters with the same label for different DLG have no relation.
gdf_ccentroids = geopandas.GeoDataFrame(
    columns=["X", "Y", "cluster", "geometry"], geometry="geometry", crs=gdf_SS.crs
)
gdf_centroids = geopandas.GeoDataFrame(
    columns=[
        "building",
        "amenity",
        "area",
        "CNIS_PROVI",
        "CNIS_REGIO",
        "CS01012023",
        "T_ARRD_NL",
        "T_MUN_NL",
        "T_NIS6_NL",
        "T_SEC_NL",
        "POPULATION",
        "cluster",
        "geometry",
    ],
    geometry="geometry",
)

cluster_offset = 0  # Offset to generate unique cluster IDs over DLG

# Itereer over de deelgemeenten
# for i_dlg in gdf_SS.sort_values(by=['T_MUN_NL', 'T_NIS6_NL'])['T_NIS6_NL'].unique():
# Sort on the names (T attributes) but group on the codes/indices (C attributes) to deal with duplicate naming
for group_k, group_v in gdf_SS.sort_values(by=["T_MUN_NL", "T_NIS6_NL"]).groupby(
    by=["CNIS5_2023", "C_NIS6"], sort=False
):
    i_gem = group_k[0]  # group_k is a tuple
    i_dlg = group_k[1]
    i_gem_name = group_v["T_MUN_NL"].iloc[
        0
    ]  # group_v is a Pandas Series of (identical) values for each member of the group, take the first element
    i_dlg_name = group_v["T_NIS6_NL"].iloc[0]
    # Determine municipality (gemeente) to which the submunicipality (deelgemeente) belongs
    # BUG : this finds multiple GEM for DLG with the same name, e.g. Halle (Halle) and Halle (Zoersel)
    #       no major issue, but double processing
    # i_gem = gdf_SS[gdf_SS["T_NIS6_NL"] == i_dlg].iloc[0]["T_MUN_NL"]

    # Switch for testing
    if True:
        #    if i_gem_name in [ 'Deurne', 'Halle', 'Diest', 'Hasselt', 'Diepenbeek']:
        # Note: Deurne only exists as DLG, not as GEM. So it should not pop up in the list
        print(f"{Fore.YELLOW}Processing {i_dlg_name} ({i_gem_name}) {Style.RESET_ALL}")

        # Create folder for DLG specific data
        os.makedirs(
            f"./data/{slugify(i_dlg_name)}_{slugify(i_gem_name)}", exist_ok=True
        )  # create data folder if not exist

        # Retrieve the number of the largest SS required to cover x_pct of the i_dlg population
        # This will be the lower bound of the number of clusters
        # (Use iloc[0] and not [0] as there is no row index starting from 0)
        n_x_pct = DLG_pivot_gdf.loc[
            DLG_pivot_gdf["C_NIS6"] == i_dlg, f"n_pct_{x_pct}"
        ].iloc[0]
        mindist = DLG_pivot_gdf.loc[DLG_pivot_gdf["C_NIS6"] == i_dlg, "mindist"].iloc[0]

        # Select the ss falling withing i_dlg
        ss_i_dlg_gdf = gdf_SS[gdf_SS["C_NIS6"] == i_dlg]

        # Only keep building centroids falling within i_dlg
        bldng_centroids_dlg = bldng_centroids.overlay(
            ss_i_dlg_gdf, how="intersection", make_valid=True
        )
        a = pd.Series(bldng_centroids_dlg["geometry"].apply(lambda p: p.x))
        b = pd.Series(bldng_centroids_dlg["geometry"].apply(lambda p: p.y))
        X = np.column_stack((a, b))

        silhouette_coefficients = {}
        n_ss = len(ss_i_dlg_gdf)  # number of statistical sectors
        print(f"Number of statistical sectors: {n_ss}")
        # We can't cluster without points (e.g. Spalbeek (Hasselt)), report and skip
        n_bldng_dlg = len(
            bldng_centroids_dlg
        )  # number of buildings in DLG (= number of samples in dataset)
        if n_bldng_dlg == 0:
            print(f"No points to cluster found in DLG. Moving on ...")
            continue
        print(f"Number of buildings in DLG: {n_bldng_dlg}")
        print(
            f"{n_x_pct} largest SS contain(s) at least {x_pct}% of the {i_dlg_name} population"
        )

        # Try different number of clusters, ranging from the amount of SS needed to cover x_pct of the population of the DLG up to
        # the number of SS in the DLG
        if mindist < 25:
            n_clust_min = n_x_pct  # Minimal amount of clusters for DLG
            n_clust_max = n_ss  # Maximal amount of clusters for DLG
        elif mindist < 50:
            n_clust_min = math.floor(n_x_pct / 2) + 1
            n_clust_max = math.floor(n_ss / 2) + 1
        else:
            n_clust_min = 1
            n_clust_max = 1
        if n_clust_max < n_clust_min:
            n_clust_min = n_clust_max

        # entry of iteration loop for feedback from previous clustering
        repeat_dlg_clustering = True
        while repeat_dlg_clustering:
            # If we have less samples than the minimal number of clusters, reduce number of clusters
            if n_clust_min > n_bldng_dlg:
                n_clust_min = n_bldng_dlg
            # If we have less samples than the maximal number of clusters, reduce number of clusters
            if n_clust_max > n_bldng_dlg:
                n_clust_max = n_bldng_dlg
            for i_n_clust in range(n_clust_min, n_clust_max + 1):
                kmeans = KMeans(
                    n_clusters=i_n_clust,
                    init="k-means++",
                    random_state=42,
                    n_init="auto",
                )
                kmeans.fit(X)

                # Initialise Silhouette score to worst possible value
                sil_coeff = -1
                if i_n_clust > 1 and i_n_clust < n_bldng_dlg:
                    # Compute Silhouette score if it can be computed (requires at least 2 clusters/labels)
                    sil_coeff = silhouette_score(X, kmeans.labels_, metric="euclidean")
                elif i_n_clust > 1 and i_n_clust == n_bldng_dlg:
                    # If the number of clusters is equal to the number of observations, we can get a perfect match
                    # but can't compute the silhouette function. Set it to max value (1)
                    sil_coeff = 1
                silhouette_coefficients[i_n_clust] = sil_coeff

                # Compute the classes for the centroids (which might be silly as they are probably the row index of kmeans.labels_, but can't find it in the documentation now. Better be safe.)
                Xcentroids = np.column_stack(
                    (kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1])
                )
                y_centroids = kmeans.predict(Xcentroids)

                print(
                    f"# clusters: {i_n_clust}, Inertia: {kmeans.inertia_}, Silhouette coefficient: {sil_coeff}"
                )

                # Create dataframe with the cluster centroids and the labels of the clusters
                df_ccentroids_tmp = pd.DataFrame(
                    {
                        "X": kmeans.cluster_centers_[:, 0],
                        "Y": kmeans.cluster_centers_[:, 1],
                        "cluster": y_centroids,
                    }
                )
                # df_ccentroids_tmp.apply(lambda row: print(row), axis=1)
                df_ccentroids_tmp["geometry"] = df_ccentroids_tmp.apply(
                    lambda row: Point(row["X"], row["Y"]), axis=1
                )
                # print(gdf_cnts.crs) # find better way to set crs
                gdf_ccentroids_tmp = gpd.GeoDataFrame(df_ccentroids_tmp).set_crs(
                    crs=gdf_cnts.crs
                )
                gdf_ccentroids_tmp.to_file(
                    f"./data/{slugify(i_dlg_name)}_{slugify(i_gem_name)}/clusters_{slugify(i_dlg_name)}_{slugify(i_gem_name)}_{i_n_clust}_centroids.shp"
                )

                # Create dataframe with classes added to the building centroids
                y_kmeans = kmeans.predict(X)
                # Todo: bovenstaand is gelijk aan kmeans.labels_ ??
                df_centroids_tmp = pd.DataFrame(y_kmeans, columns=["cluster"])
                df_centroids_tmp = df_centroids_tmp.assign(n_ss=n_ss)
                gdf_centroids_tmp = bldng_centroids_dlg.join(df_centroids_tmp)
                gdf_centroids_tmp.to_file(
                    f"./data/{slugify(i_dlg_name)}_{slugify(i_gem_name)}/clusters_{slugify(i_dlg_name)}_{slugify(i_gem_name)}_{i_n_clust}.shp"
                )

            # Get the smallest number of clusters (dict key) that has the highest sihouette function value (dict value)
            optimal_nr_clusters = max(
                sorted(silhouette_coefficients), key=silhouette_coefficients.get
            )
            print(
                f"Optimal number of clusters: {Fore.RED}{optimal_nr_clusters}{Style.RESET_ALL}"
            )

            # Now we have found the optimal number of clusters, rerun the clustering to test and export the data
            kmeans = KMeans(
                n_clusters=optimal_nr_clusters,
                init="k-means++",
                random_state=42,
                n_init="auto",
            )
            kmeans.fit(X)
            y_kmeans = kmeans.predict(X) + cluster_offset

            # Test whether the clusters found are acceptable
            df_centroids_tmp = pd.DataFrame(y_kmeans, columns=["cluster"])
            gdf_centroids_tmp = bldng_centroids_dlg.join(df_centroids_tmp)

            # compute the convex hull of a geoseries (of points); this is used to compute the convex hull of the cluster
            # convex_hull on a geoseries is the geoseries of the convex hulls
            # so we first have to union the points in our geoseries before computing the convex hull
            def clustarea(x):
                return x.unary_union.convex_hull.area

            gdf_cluster_polygons = gdf_centroids_tmp.pivot_table(
                index="cluster", values=["geometry"], aggfunc={"geometry": [clustarea]}
            )
            # Flatten multi level column index
            gdf_cluster_polygons.columns = [
                a[-1] for a in gdf_cluster_polygons.columns.to_flat_index()
            ]
            # Set constraints on the size of the clusters (to manage travel time within the cluster)
            max_cluster_area = 1000000000
            if mindist < 25:
                max_cluster_area = math.pi * math.pow(
                    1000, 2
                )  # circle with 1000m radius
            elif mindist < 50:
                max_cluster_area = math.pi * math.pow(
                    2500, 2
                )  # circle with 2500m radius
            else:
                max_cluster_area = math.pi * math.pow(
                    5000, 2
                )  # circle with 5000m radius

            n_too_large = len(
                gdf_cluster_polygons.loc[
                    gdf_cluster_polygons["clustarea"] > max_cluster_area
                ]
            )
            if n_too_large > 0:
                # Increase number of clusters with 1
                optimal_nr_clusters += 1
                silhouette_coefficients = {}  # Reset silhouette coefficients
                print(
                    f"{n_too_large} clusters have an area larger than ({round(max_cluster_area)}). Increasing # clusters to {optimal_nr_clusters}"
                )
                n_clust_min = optimal_nr_clusters
                n_clust_max = optimal_nr_clusters
                repeat_dlg_clustering = True  # recluster
            else:
                repeat_dlg_clustering = False  # Stop iteration

        # Clusters found for the DLG are ok, export
        df_centroids_tmp = pd.DataFrame(y_kmeans, columns=["cluster"])
        gdf_centroids_tmp = bldng_centroids_dlg.join(df_centroids_tmp)
        gdf_centroids_tmp.to_file(
            f"./data/{slugify(i_dlg_name)}_{slugify(i_gem_name)}/clusters_{slugify(i_dlg_name)}_{slugify(i_gem_name)}_optimal.shp"
        )

        # Add centroids to global layer with labeled building centroids
        # Added check if dataframes are empty to avoid deprecation warning for concat of empty dataframes
        gdf_centroids = (
            gdf_centroids_tmp
            if gdf_centroids.empty
            else pd.concat([gdf_centroids, gdf_centroids_tmp])
        )

        # Compute the classes for the centroids (which might be silly as they are probably the row index of kmeans.labels_, but can't find it in the documentation now. Better be safe.)
        Xcentroids = np.column_stack(
            (kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1])
        )
        y_centroids = kmeans.predict(Xcentroids)
        # Create dataframe with the cluster centroids and the labels of the clusters
        df_ccentroids_tmp = pd.DataFrame(
            {
                "X": kmeans.cluster_centers_[:, 0],
                "Y": kmeans.cluster_centers_[:, 1],
                "cluster": y_centroids + cluster_offset,
            }
        )
        df_ccentroids_tmp["geometry"] = df_ccentroids_tmp.apply(
            lambda row: Point(row["X"], row["Y"]), axis=1
        )
        # todo: find better way to set crs
        gdf_ccentroids_tmp = gpd.GeoDataFrame(df_ccentroids_tmp).set_crs(
            crs=gdf_cnts.crs
        )
        gdf_ccentroids_tmp.to_file(
            f"./data/{slugify(i_dlg_name)}_{slugify(i_gem_name)}/clusters_{slugify(i_dlg_name)}_{slugify(i_gem_name)}_optimal_centroids.shp"
        )

        # Add ccentroids to global layer with cluster centroids
        # Added check if dataframes are empty to avoid deprecation warning for concat of empty dataframes
        gdf_ccentroids = (
            gdf_ccentroids_tmp
            if gdf_ccentroids.empty
            else pd.concat([gdf_ccentroids, gdf_ccentroids_tmp])
        )

        # Update cluster offset
        cluster_offset += optimal_nr_clusters

# Write all centroids to file
gdf_centroids.to_file(f"./data/clusters_optimal.shp")
# Write all ccentroids to file
gdf_ccentroids.to_file(f"./data/clusters_optimal_centroids.shp")